#### Check De Data

In [31]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
import dvc.api
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_hastie_10_2
from sklearn.preprocessing import LabelEncoder
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from IPython.display import display

import logging

In [32]:
path="../data/prepared/beer_profile_and_ratings.csv"

In [33]:
with open(path,'rb') as f:
    df = pd.read_csv(f)
    display(df.head())
    print(df.columns)
    
    df_preprocess = df[['Name', 'Style', 'ABV',
       'Min IBU', 'Max IBU', 'Astringency', 'Body', 'Alcohol', 'Bitter',
       'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices', 'Malty',
       'review_aroma', 'review_appearance', 'review_palate', 'review_taste',
       'review_overall']]
    df_preprocess.set_index('Name')
    df_preprocess.to_csv("../data/prepared/beer_profile_and_ratings.csv", index=False)
    display(df_preprocess.head())
   

,Name,Style,ABV,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,...,Salty,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall
0,Amber,Altbier,5.3,25,50,13,32,9,47,74,...,0,33,57,8,111,3.498994,3.636821,3.556338,3.643863,3.847082
1,Double Bag,Altbier,7.2,25,50,12,57,18,33,55,...,0,24,35,12,84,3.798337,3.846154,3.904366,4.024948,4.034304
2,Long Trail Ale,Altbier,5.0,25,50,14,37,6,42,43,...,0,10,54,4,62,3.409814,3.667109,3.600796,3.631300,3.830239
3,Doppelsticke,Altbier,8.5,25,50,13,55,31,47,101,...,1,49,40,16,119,4.148098,4.033967,4.150815,4.205163,4.005435
4,Sleigh'r Dark Doüble Alt Ale,Altbier,7.2,25,50,25,51,26,44,45,...,1,11,51,20,95,3.625000,3.973958,3.734375,3.765625,3.817708


Index(['Name', 'Style', 'ABV', 'Min IBU', 'Max IBU', 'Astringency', 'Body',
       'Alcohol', 'Bitter', 'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy',
       'Spices', 'Malty', 'review_aroma', 'review_appearance', 'review_palate',
       'review_taste', 'review_overall'],
      dtype='object')


,Name,Style,ABV,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,...,Salty,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall
0,Amber,Altbier,5.3,25,50,13,32,9,47,74,...,0,33,57,8,111,3.498994,3.636821,3.556338,3.643863,3.847082
1,Double Bag,Altbier,7.2,25,50,12,57,18,33,55,...,0,24,35,12,84,3.798337,3.846154,3.904366,4.024948,4.034304
2,Long Trail Ale,Altbier,5.0,25,50,14,37,6,42,43,...,0,10,54,4,62,3.409814,3.667109,3.600796,3.631300,3.830239
3,Doppelsticke,Altbier,8.5,25,50,13,55,31,47,101,...,1,49,40,16,119,4.148098,4.033967,4.150815,4.205163,4.005435
4,Sleigh'r Dark Doüble Alt Ale,Altbier,7.2,25,50,25,51,26,44,45,...,1,11,51,20,95,3.625000,3.973958,3.734375,3.765625,3.817708


In [54]:
with open(path,'rb') as f:
    df = pd.read_csv(f, index_col="Name")
    display(df.head())
    le = LabelEncoder()
    
    display(df[["Style"]].values.ravel())
    # encoded = le.fit_transform(df.Style.to_list())
    # display(encoded.reshape(-1,1))

    

,Style,ABV,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall
Name,,,,,,,,,,,,,,,,,,,,
Amber,Altbier,5.3,25,50,13,32,9,47,74,33,0,33,57,8,111,3.498994,3.636821,3.556338,3.643863,3.847082
Double Bag,Altbier,7.2,25,50,12,57,18,33,55,16,0,24,35,12,84,3.798337,3.846154,3.904366,4.024948,4.034304
Long Trail Ale,Altbier,5.0,25,50,14,37,6,42,43,11,0,10,54,4,62,3.409814,3.667109,3.600796,3.631300,3.830239
Doppelsticke,Altbier,8.5,25,50,13,55,31,47,101,18,1,49,40,16,119,4.148098,4.033967,4.150815,4.205163,4.005435
Sleigh'r Dark Doüble Alt Ale,Altbier,7.2,25,50,25,51,26,44,45,9,1,11,51,20,95,3.625000,3.973958,3.734375,3.765625,3.817708


array(['Altbier', 'Altbier', 'Altbier', ..., 'Winter Warmer',
       'Winter Warmer', 'Winter Warmer'], dtype=object)

#### Check data thruough dvc pipe

In [78]:
path="data/prepared/beer_profile_and_ratings.csv"
repo="https://github.com/stijnhering/PreTraineeship"
version="<GIT COMMIT>"
remote="storage"

In [79]:
data_url = dvc.api.get_url(path=path, repo=repo)
print(data_url)

gdrive://18YK670W5X0QSNCNshMXTme1Ff7fzWBo9/89/791ee3d014a17e17e464328e217447


In [80]:
with dvc.api.open(path, 
                  repo=repo) as fd:
        df = pd.read_csv(fd)
        display(df)
    

/home/smwhering/miniconda3/envs/dataeng/lib/python3.9/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access /tmp/tmp3am5r89gdvc-clone/.dvc/tmp/gdrive-user-credentials.json: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))


Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=710796635688-iivsgbgsb6uv1fap6635dhvuei09o66c.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.appdata&access_type=offline&response_type=code&approval_prompt=force

Authentication successful.


,Name,Style,ABV,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,...,Salty,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall
0,Amber,Altbier,5.3,25,50,13,32,9,47,74,...,0,33,57,8,111,3.498994,3.636821,3.556338,3.643863,3.847082
1,Double Bag,Altbier,7.2,25,50,12,57,18,33,55,...,0,24,35,12,84,3.798337,3.846154,3.904366,4.024948,4.034304
2,Long Trail Ale,Altbier,5.0,25,50,14,37,6,42,43,...,0,10,54,4,62,3.409814,3.667109,3.600796,3.631300,3.830239
3,Doppelsticke,Altbier,8.5,25,50,13,55,31,47,101,...,1,49,40,16,119,4.148098,4.033967,4.150815,4.205163,4.005435
4,Sleigh'r Dark Doüble Alt Ale,Altbier,7.2,25,50,25,51,26,44,45,...,1,11,51,20,95,3.625000,3.973958,3.734375,3.765625,3.817708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,Winter Shredder,Winter Warmer,8.8,35,50,15,37,24,35,46,...,0,23,45,67,74,4.125000,3.875000,3.875000,3.750000,4.000000
3193,The First Snow Ale,Winter Warmer,6.0,35,50,15,31,23,16,54,...,0,54,14,140,58,3.950000,3.750000,3.760000,3.790000,3.730000
3194,Red Nose Winter Ale,Winter Warmer,6.8,35,50,8,44,24,19,52,...,0,26,21,96,77,3.576923,3.711538,3.596154,3.673077,3.711538
3195,Fish Tale Winterfish,Winter Warmer,7.5,35,50,11,36,50,70,72,...,0,81,110,18,73,3.902299,3.885057,3.862069,3.902299,3.879310
